# PO : Optimization of Complex Decisions - 
# Project : Optimization of the planning 

### Sets

 $H$ = Time window

 $S$ = Staffing set

 $Q$ = Qualification set

$P$ = Project set
### Parameters

A. Employees

-  $C = (c_{s,q})_{(s,q)\in S \times Q}$     = Matrix of qualifications
-  $G = (g_{t,s})_{(t,s)\in H \times S}$     = Matrix of days off

B. Projects

- $N = (n_{q,p})_{(q,p)\in Q \times P}$ = Matrix of worked days by qualification by project
- $CA = (ca_{p})_{p\in P}$ = Matrix of values for each project
- $D = (d_{p,h})_{(p,h)\in P \times H}$ = Matrix of the deadlines
- $R = (r_p)_{p \in P}$ = Matrix of Penalty 



### Variables

A. Working day
- $w_{t,p,q,s} \in {0,1}$ Is 1 if staff member s is working on day t on the qualification q of project p
- $y_p \in {{0,1}}$ Is 1 if project p is finished in the time window
- $z_p \in \mathbb{N}$ Completion date
- $l_p \in \mathbb{N}$ Completion date - due date. 0 if completed earlier
- $a_{s,p} \in {0,1}$ Is 1 if employee s is working on project p anytime



### Objective

We try to maximize the profit
$\max_{w_{t,p,q,s},y_p \in ({0,1}^2)} Z = \sum_{p\in P}y_pca_p -l_pr_p$

### Constraints
- $w_{t,p,q,s} \leq c_{s,q}$ : We can't work for a qualification if we don't have it
- $\sum_{p,q}w_{t,p,q,s} \leq 1$ : A worker can't work on 2 projects nor 2 qualifications at the same time
- $w_{t,p,q,s} \leq g_{t,s}$ : A worker can't work on his day off
- $\forall q \in Q  \sum{w_{t,p,q,s}} \geq n_{p,q}y_p$ We must have worked all the requiered days in all the qualifications to finish a project
- $ \forall p \in P \space z_p \geq t*w_{t,p,q,s}$ The completion date is the last time someone worked on a project
- $ \forall p \in P \space z_p \geq (h-1)*(1-y_p)$ If not completed the penalty is maxed
- $ \forall p \in P \space l_p \geq z_p - d_p$ We get the value of the delay
- $\forall s \in S \space \sum_{p\in P}a_{s,p} \leq \epsilon$ Epsilon constraint to minimize the number of projects worked on by each employee